# Get King Taco locations

#### Load Python tools and Jupyter config

In [1]:
import us
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
from bs4 import BeautifulSoup
from vega_datasets import data as vega_data
from tqdm.notebook import tqdm, trange

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
place = "king-taco"
place_formal = "King Taco"
color = "#FD4D2D"
today = pd.Timestamp.today().strftime("%Y-%m-%d")

---

## Scrape

#### Headers for request

In [4]:
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

In [5]:
params = {
    "action": "store_search",
    "lat": "33.98896",
    "lng": "-118.4165",
    "max_results": "100",
    "search_radius": "1000",
    "skip_cache": "1",
}

response = requests.get(
    "https://kingtaco.com/wp-admin/admin-ajax.php", params=params, headers=headers
)

In [6]:
dict_list = response.json()

In [7]:
src = pd.DataFrame(dict_list)

In [8]:
src["state"] = src["state"].str.upper()

In [9]:
df = src[
    [
        "id",
        "store",
        "address",
        "city",
        "state",
        "zip",
        "lat",
        "lng",
        "phone",
    ]
].copy()

In [10]:
df

,id,store,address,city,state,zip,lat,lng,phone
0,292,KING TACO #1,1118 Cypress Ave,Los Angeles,CA,90065,34.10748,-118.22577,(323) 223-2595
1,294,KING TACO #2,4504 East 3rd Street,Los Angeles,CA,90022,34.033048,-118.168814,(323) 264-4067
2,296,KING TACO #6A,1951 S. San Pedro Street,Los Angeles,CA,90011,34.026618,-118.258624,(213) 747-3176
3,887,KING TACO #6B,645 E. Washington Blvd.,Los Angeles,CA,90015,34.02826,-118.25745,(213) 747-9915
4,890,KING TACO #8,2904 N. Broadway Ave,Los Angeles,CA,90031,34.07367,-118.21142,(323) 222-8500
5,893,KING TACO #9,2400 E. Cesar Chavez Ave,Los Angeles,CA,90033,34.04667,-118.20750,(323) 264-3940
6,896,KING TACO #10,2020 W. Pico Boulevard,Los Angeles,CA,90006,34.04695,-118.28209,
7,899,KING TACO #11,10420 E Garvey Ave,El Monte,CA,91733,34.06279,-118.04452,(626) 448-5150
8,902,KING TACO #12,5729 Atlantic Blvd.,Maywood,CA,90270,33.98848,-118.18565,(323) 560-1245
9,905,KING TACO #15,4300 E. Olympic Blvd,Los Angeles,CA,90023,34.01878,-118.17966,(323) 264-9051


#### Create a mapping of state abbreviations to full state names using the us library

In [11]:
state_mapping = {state.abbr: state.name for state in us.states.STATES}

#### New column of full state names based on abbreviations

In [12]:
df["state_name"] = df["state"].map(state_mapping)

#### Make sure our brand name gets in the dataframe

In [13]:
df["brand"] = place_formal

#### Add fetch date

In [14]:
df["updated"] = today

---

## Geography

#### Make it a geodataframe

In [15]:
df_geo = df.copy()

In [16]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.lng, df_geo.lat)
).set_crs("4326")

---

## Maps

#### US states background

In [24]:
background = (
    alt.Chart(alt.topo_feature(vega_data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [25]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=5, color=color)
    .encode(
        longitude="lng:Q",
        latitude="lat:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

## Exports

#### JSON

In [27]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [28]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [29]:
gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)